In [1]:
!apt-get update
!apt-get install -y libopencv-dev
!pip install opencv-python

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,978 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,566 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-up

In [2]:
%%writefile meanFilter.cu
#include <stdio.h>
#include <stdlib.h>
#include <opencv2/opencv.hpp>
#include <cuda_runtime.h>
using namespace cv;

__global__ void meanFilter(const uchar3* input, uchar3* output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int filterSize = 7;
    int offset = filterSize / 2;

    if (x >= offset && y >= offset && x < width - offset && y < height - offset) {
        int3 sum = make_int3(0, 0, 0);
        for (int fy = -offset; fy <= offset; fy++) {
            for (int fx = -offset; fx <= offset; fx++) {
                uchar3 val = input[(y + fy) * width + (x + fx)];
                sum.x += val.x;
                sum.y += val.y;
                sum.z += val.z;
            }
        }
        int area = filterSize * filterSize;
        output[y * width + x] = make_uchar3(sum.x / area, sum.y / area, sum.z / area);
    }
}

int main(int argc, char** argv) {
    if (argc != 2) {
        printf("usage: DisplayImage.out <Image_Path>\n");
        return -1;
    }

    Mat image = imread(argv[1], IMREAD_COLOR);
    if (image.empty()) {
        printf("Could not open or find the image\n");
        return -1;
    }

    int height = image.rows;
    int width = image.cols;
    size_t sizeInBytes = width * height * sizeof(uchar3);

    printf("Slika je veličine %i x %i\n", height, width);

    uchar3 *d_input, *d_output;
    cudaMalloc(&d_input, sizeInBytes);
    cudaMalloc(&d_output, sizeInBytes);

    cudaMemcpy(d_input, image.ptr<uchar3>(0), sizeInBytes, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((width + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (height + threadsPerBlock.y - 1) / threadsPerBlock.y);

    meanFilter<<<numBlocks, threadsPerBlock>>>(d_input, d_output, width, height);
    cudaDeviceSynchronize();

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
        return -1;
    }

    uchar3* h_output = (uchar3*)malloc(sizeInBytes);
    cudaMemcpy(h_output, d_output, sizeInBytes, cudaMemcpyDeviceToHost);

    Mat image_out(height, width, CV_8UC3, h_output);
    imwrite("mean_filtered.jpg", image_out);

    cudaFree(d_input);
    cudaFree(d_output);
    free(h_output);

    printf("Filtrirana slika je spremljena kao 'mean_filtered.jpg'\n");

    return 0;
}

Writing meanFilter.cu


In [3]:
!nvcc -o meanFilter meanFilter.cu $(pkg-config --cflags --libs opencv4) -diag-suppress=611

In [5]:
!./meanFilter Brela3.jpg

Slika je veličine 720 x 1240
Filtrirana slika je spremljena kao 'mean_filtered.jpg'
